In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MS_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3506337207,MS,"238 COURTHOUSE RD, GULFPORT, MS",30.381123,-89.044000,2.804700e+14
1,8685327005,MS,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS",32.446636,-90.151970,2.808903e+14
2,5587558700,MS,"109 S Broadway St, Tupelo, MS",34.257282,-88.705376,2.808195e+14
3,8446217010,MS,"4374-A MANGUM DR, FLOWOOD, MS",32.319440,-90.107650,2.812102e+14
4,3918387109,MS,"600 FRANKLIN DR, HOUSTON, MS",33.914646,-89.016710,2.801795e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3506337207,MS,"238 COURTHOUSE RD, GULFPORT, MS",30.381123,-89.044000,2.804700e+14
1,8685327005,MS,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS",32.446636,-90.151970,2.808903e+14
2,5587558700,MS,"109 S Broadway St, Tupelo, MS",34.257282,-88.705376,2.808195e+14
3,8446217010,MS,"4374-A MANGUM DR, FLOWOOD, MS",32.319440,-90.107650,2.812102e+14
4,3918387109,MS,"600 FRANKLIN DR, HOUSTON, MS",33.914646,-89.016710,2.801795e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
314,2980208300,MS,"1109 HIGHLAND COLONY PARKWAY SUITE 219, RIDGEL...",32.444901,-90.152214,NaN
466,9091747108,MS,"11751 HIGHWAY 45, MACON, MS",37.641395,-88.676502,NaN
848,4954377010,MS,"484 HWY. 145 S, BOONEVILLE, MS",35.560405,-91.176158,NaN
1069,4600107209,MS,"736 HICKMAN ST, WIGGINS, MS",30.042524,-89.989323,NaN
1100,6485998609,MS,"736 Hickman St, Wiggins, MS",30.042524,-89.989323,NaN
1917,4426007109,MS,"4875 I 55 NORTH SUITE 210, JACKSON, MS",35.220412,-90.209775,NaN
2698,4481087103,MS,"4785 I 55 North Suite 200, JACKSON, MS",35.220412,-90.209775,NaN
3702,6319687107,MS,"22019 HIGHWAY 45 N, ABERDEEN, MS",42.232892,-87.987392,NaN
3793,6171907006,MS,"15029 US HIGHWAY 45, MACON, MS",36.586400,-88.797449,NaN
3879,5276427300,MS,"16814 US HIGHWAY 49, BELZONI, MS",34.800155,-91.188100,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('28')] #MS
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/23 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

23


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_11998/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
314,2980208300,MS,"1109 HIGHLAND COLONY PARKWAY SUITE 219, RIDGEL...",32.444901,-90.152214,[280890302051004]
466,9091747108,MS,"11751 HIGHWAY 45, MACON, MS",37.641395,-88.676502,[]
848,4954377010,MS,"484 HWY. 145 S, BOONEVILLE, MS",35.560405,-91.176158,[]
1069,4600107209,MS,"736 HICKMAN ST, WIGGINS, MS",30.042524,-89.989323,[]
1100,6485998609,MS,"736 Hickman St, Wiggins, MS",30.042524,-89.989323,[]
1917,4426007109,MS,"4875 I 55 NORTH SUITE 210, JACKSON, MS",35.220412,-90.209775,[]
2698,4481087103,MS,"4785 I 55 North Suite 200, JACKSON, MS",35.220412,-90.209775,[]
3702,6319687107,MS,"22019 HIGHWAY 45 N, ABERDEEN, MS",42.232892,-87.987392,[]
3793,6171907006,MS,"15029 US HIGHWAY 45, MACON, MS",36.586400,-88.797449,[]
3879,5276427300,MS,"16814 US HIGHWAY 49, BELZONI, MS",34.800155,-91.188100,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_11998/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
314,2980208300,MS,"1109 HIGHLAND COLONY PARKWAY SUITE 219, RIDGEL...",32.444901,-90.152214,280890302051004
466,9091747108,MS,"11751 HIGHWAY 45, MACON, MS",37.641395,-88.676502,NaN
848,4954377010,MS,"484 HWY. 145 S, BOONEVILLE, MS",35.560405,-91.176158,NaN
1069,4600107209,MS,"736 HICKMAN ST, WIGGINS, MS",30.042524,-89.989323,NaN
1100,6485998609,MS,"736 Hickman St, Wiggins, MS",30.042524,-89.989323,NaN
1917,4426007109,MS,"4875 I 55 NORTH SUITE 210, JACKSON, MS",35.220412,-90.209775,NaN
2698,4481087103,MS,"4785 I 55 North Suite 200, JACKSON, MS",35.220412,-90.209775,NaN
3702,6319687107,MS,"22019 HIGHWAY 45 N, ABERDEEN, MS",42.232892,-87.987392,NaN
3793,6171907006,MS,"15029 US HIGHWAY 45, MACON, MS",36.586400,-88.797449,NaN
3879,5276427300,MS,"16814 US HIGHWAY 49, BELZONI, MS",34.800155,-91.188100,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
466,9091747108,MS,"11751 HIGHWAY 45, MACON, MS",37.641395,-88.676502,NaN
848,4954377010,MS,"484 HWY. 145 S, BOONEVILLE, MS",35.560405,-91.176158,NaN
1069,4600107209,MS,"736 HICKMAN ST, WIGGINS, MS",30.042524,-89.989323,NaN
1100,6485998609,MS,"736 Hickman St, Wiggins, MS",30.042524,-89.989323,NaN
1917,4426007109,MS,"4875 I 55 NORTH SUITE 210, JACKSON, MS",35.220412,-90.209775,NaN
2698,4481087103,MS,"4785 I 55 North Suite 200, JACKSON, MS",35.220412,-90.209775,NaN
3702,6319687107,MS,"22019 HIGHWAY 45 N, ABERDEEN, MS",42.232892,-87.987392,NaN
3793,6171907006,MS,"15029 US HIGHWAY 45, MACON, MS",36.586400,-88.797449,NaN
3879,5276427300,MS,"16814 US HIGHWAY 49, BELZONI, MS",34.800155,-91.188100,NaN
4098,1995067101,MS,"5385 I 55 N, JACKSON, MS",35.267050,-90.230373,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
314,2980208300,280890302051004
466,9091747108,NaN
848,4954377010,NaN
1069,4600107209,NaN
1100,6485998609,NaN
1917,4426007109,NaN
2698,4481087103,NaN
3702,6319687107,NaN
3793,6171907006,NaN
3879,5276427300,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MS_fips_scraped.csv") 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,512795,3506337207,MS,"238 COURTHOUSE RD, GULFPORT, MS",30.381123,-89.044000,2.804700e+14
1,512796,8685327005,MS,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS",32.446636,-90.151970,2.808903e+14
2,512797,5587558700,MS,"109 S Broadway St, Tupelo, MS",34.257282,-88.705376,2.808195e+14
3,512798,8446217010,MS,"4374-A MANGUM DR, FLOWOOD, MS",32.319440,-90.107650,2.812102e+14
4,512799,3918387109,MS,"600 FRANKLIN DR, HOUSTON, MS",33.914646,-89.016710,2.801795e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,512795,3506337207,MS,"238 COURTHOUSE RD, GULFPORT, MS",30.381123,-89.044000,2.804700e+14,NaN
1,512796,8685327005,MS,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS",32.446636,-90.151970,2.808903e+14,NaN
2,512797,5587558700,MS,"109 S Broadway St, Tupelo, MS",34.257282,-88.705376,2.808195e+14,NaN
3,512798,8446217010,MS,"4374-A MANGUM DR, FLOWOOD, MS",32.319440,-90.107650,2.812102e+14,NaN
4,512799,3918387109,MS,"600 FRANKLIN DR, HOUSTON, MS",33.914646,-89.016710,2.801795e+14,NaN
...,...,...,...,...,...,...,...,...
5499,518294,6662018610,MS,"300 Heritage Dr, Oxford, MS",34.367306,-89.560310,2.807195e+14,NaN
5500,518295,7192318409,MS,"6229 Highway 305 N Ste O, Olive Branch, MS",34.952320,-89.830414,2.803307e+14,NaN
5501,518296,7483017308,MS,"41033 OLD HIGHWAY 45 South, ABERDEEN, MS",33.810067,-88.477517,2.809595e+14,NaN
5502,518297,7726238600,MS,"116 Price St, McComb, MS",31.247960,-90.469820,2.811395e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,512795,3506337207,MS,"238 COURTHOUSE RD, GULFPORT, MS",30.381123,-89.044000,2.804700e+14,NaN,280470014022015.0
1,512796,8685327005,MS,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS",32.446636,-90.151970,2.808903e+14,NaN,280890302051012.0
2,512797,5587558700,MS,"109 S Broadway St, Tupelo, MS",34.257282,-88.705376,2.808195e+14,NaN,280819507001030.0
3,512798,8446217010,MS,"4374-A MANGUM DR, FLOWOOD, MS",32.319440,-90.107650,2.812102e+14,NaN,281210203041000.0
4,512799,3918387109,MS,"600 FRANKLIN DR, HOUSTON, MS",33.914646,-89.016710,2.801795e+14,NaN,280179503002008.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3506337207,MS,"238 COURTHOUSE RD, GULFPORT, MS",30.381123,-89.044000,280470014022015.0
1,8685327005,MS,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS",32.446636,-90.151970,280890302051012.0
2,5587558700,MS,"109 S Broadway St, Tupelo, MS",34.257282,-88.705376,280819507001030.0
3,8446217010,MS,"4374-A MANGUM DR, FLOWOOD, MS",32.319440,-90.107650,281210203041000.0
4,3918387109,MS,"600 FRANKLIN DR, HOUSTON, MS",33.914646,-89.016710,280179503002008.0
...,...,...,...,...,...,...
5499,6662018610,MS,"300 Heritage Dr, Oxford, MS",34.367306,-89.560310,280719503042000.0
5500,7192318409,MS,"6229 Highway 305 N Ste O, Olive Branch, MS",34.952320,-89.830414,280330708213005.0
5501,7483017308,MS,"41033 OLD HIGHWAY 45 South, ABERDEEN, MS",33.810067,-88.477517,280959505022072.0
5502,7726238600,MS,"116 Price St, McComb, MS",31.247960,-90.469820,281139504002077.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
466,9091747108,MS,"11751 HIGHWAY 45, MACON, MS",37.641395,-88.676502,NaN
848,4954377010,MS,"484 HWY. 145 S, BOONEVILLE, MS",35.560405,-91.176158,NaN
1069,4600107209,MS,"736 HICKMAN ST, WIGGINS, MS",30.042524,-89.989323,NaN
1100,6485998609,MS,"736 Hickman St, Wiggins, MS",30.042524,-89.989323,NaN
1917,4426007109,MS,"4875 I 55 NORTH SUITE 210, JACKSON, MS",35.220412,-90.209775,NaN
2698,4481087103,MS,"4785 I 55 North Suite 200, JACKSON, MS",35.220412,-90.209775,NaN
3702,6319687107,MS,"22019 HIGHWAY 45 N, ABERDEEN, MS",42.232892,-87.987392,NaN
3793,6171907006,MS,"15029 US HIGHWAY 45, MACON, MS",36.586400,-88.797449,NaN
3879,5276427300,MS,"16814 US HIGHWAY 49, BELZONI, MS",34.800155,-91.188100,NaN
4098,1995067101,MS,"5385 I 55 N, JACKSON, MS",35.267050,-90.230373,NaN


Convert to File

In [19]:
fips_MS = fips_merge_drop

In [20]:
fips_MS.to_csv("../../../data/state_data/geo/geo_fips/23/MS_fips_scraped.csv") 